## Setup

In [ ]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# check python version - you need python 3.7 or greater to run polars
!python -V

In [ ]:
# install polars
!pip install polars

In [ ]:
# import polars
import polars as pl

In [ ]:
# other useful imports for this workshop
import pandas as pd
from pathlib import Path

## Data download link
Download the zip file, extract it and then rename the csv file to `flights.csv` and store it inside your data folder

In [ ]:
# flights --- https://github.com/kevinheavey/modern-polars/blob/master/data/flights.csv.zip

## Polars vs Pandas

Polars is a DataFrame library completely written in Rust and is built to empower Python developers with a scalable and efficient framework for handling data and is considered as an alternative to the very popular pandas library.<br><br>
**Advantages of Polars over Pandas**:
1. **Speed and Performance** - Polars is embarassingly parallel and is memory optimized
2. **Lazy evaluation** - Polars incorporates lazy evaluation, which involves examining and optimizing queries to enhance their performance and minimize memory consumption. 
3. **Expressive syntax** - Polars employs a concise and intuitive syntax, making it easy to learn and use. 
4. **Method chaining** - DataFrame operations in Polars can be chained together, enabling efficient and concise data transformations.

We will explore some of these features in today's workshop.

## The Basics

### Loading a Dataset in Polars

In [ ]:
%%timeit -n 3
# Let's load some flight data - see the 'flights_readme.html' file for more details
data_path = 'data/flights.csv'

# polars code
pl.Config.set_tbl_rows(5) # don't print too many rows in the book
df_pl = pl.read_csv(data_path)

In [ ]:
df_pl

Notice that polars shows you the shape of the dataframe at the top left

In [ ]:
%%timeit -n 3
# pandas code
pd.options.display.max_rows = 5 
df_pd = pd.read_csv(data_path)

In [ ]:
df_pd

You can already see how much faster polars is at reading the csv file than pandas!

### Data exploration

A lot of the standard pandas commands such as `head` , `tail` , `describe` work as expected with a little extra output sprinkled in:

In [ ]:
df_pl.head() # polars

In [ ]:
df_pl.describe() # polars

In [ ]:
df_pd.describe() # pandas

The polars output has a couple of notable features:
* The shape is included which is useful to make sure you’re not dropping rows or columns inadvertently
* Underneath each column name is a data type which is another useful reminder
* There are no index numbers
* The string columns include ” ” around the values

### Converting between polars and pandas

As you work with pandas and polars there are convenience functions for moving back and forth between the two. Here’s an example of creating a pandas dataframe from polars:
* df_pl`.to_pandas()` will convert your polars df to pandas
* `pl.from_pandas(pd_df)` will convert pandas df to polars

Note: The conversion requires installing pyarrow and other dependencies

## Selecting and filtering rows and columns

Polars does not use indexing, unlike pandas <br>
Polars introduces the concept of Expressions to help you work with your data. There are four main expressions you need to understand when working with data in polars:
* `select` to choose the subset of columns you want to work with
* `filter` to choose the subset of rows you want to work with
* `with_columns` to create new columns
* `group_by` to group data together

Choosing or reordering columns is straightforward with `select() `

### Select

In [ ]:
# select the columns you want to keep
df_pl.select(pl.col("FlightDate", "Origin", "Dest"))

The `pl.col()` code is used to create column expressions. (See Documentation - https://docs.pola.rs/py-polars/html/reference/expressions/col.html)

### Filter

In [ ]:
# filtering 
df_pl.filter(pl.col("Origin") == "ORD")

### Add columns
The easiest way to think about it is that any time you want to add a new column to your data, you need to use `with_columns` .

In [ ]:
df_pl.with_columns(
    (pl.col("DepDelayMinutes") / 60)
    .alias("DepDelayHours")
)

This command does a couple of things to create a new column:
* Select the DepDelayMinutes column
* Do a math operation to convert time to hours
* Name the newly created column DepDelayHours using the `alias` function

### Conditional data expressions
Polars has a useful function `when` `then` `otherwise` which can replace pandas `mask` or `np.where `

In [ ]:
df_pl.with_columns(
    pl.when(pl.col("DepDelayMinutes") > 30)
    .then(pl.lit("Delayed"))
    .otherwise(pl.lit("OnTime"))
    .alias("Flight_delay_status")
)

This is somewhat analogous to an if-then-else statement in python.<br>
This example also introduces `pl.lit()` which we use to assign a literal value to the columns.

Another version of the above expression where the delay in minutes is reported if less than 30

In [ ]:
df_pl.with_columns(
    pl.when(pl.col("DepDelayMinutes") > 30)
    .then(pl.lit("Delayed"))
    .otherwise(pl.col("DepDelayMinutes"))
    .alias("Flight_delay_status")
)

### Exercises
Load the dataset below

In [ ]:
df = pl.read_csv('data/diamonds.csv')
df

#### Select specific columns: carat, cut, and price

#### Filter in rows where Carat Weight is larger than 2

#### Add a column called price_1000 which shows the price in $1000 increments
Example: if the price is 11791.57 it should show as 11.79157

#### Add a column called price_tier such that:
if the price is less than 10000, it shows the price value, else if the price is above 10000 it shows "high_range"

## Method chaining


Many languages make it easy to write `thing.min().abs().str()` instead of `str(abs(min(thing)))`. The former is called method chaining and makes for more fluent code. You've already been using method chaining but let's look at a simple example

In [ ]:
# Let's see an example with the diamonds dataset above
(df
 .filter(pl.col("Carat Weight") > 1.2)
 .groupby("Color")
 .agg(pl.mean("Price"))
)

This code reads almost like an English sentence!

### Grouping Data

The pandas `groupby` and polars `group_by` functional similarly but the key difference is that polars does not have the concept of an index or multi-index.<br>
Here’s a simple polars `group_by` example to calculate the mean flight delay by day of the week and origin.

In [ ]:
(df_pl
 .group_by("DayOfWeek", "Origin")
 .agg(pl.col("DepDelayMinutes")
      .mean()
      .alias("mean_delay"))
 .sort("mean_delay")
)

### Exercises


#### Group and chain - diamonds
Find the mean price of a diamond by color and sort the result by mean price

In [ ]:
df = pl.read_csv('data/diamonds.csv')

#### Group and chain - flights
Find the average delay in minutes for flights originating from individual cities within the state of IL. Which city has the least delay? Which city has the most delay?<br>
Hint: use the `OriginState` and `OriginCityName` columns 